# The Models

---

In an attempt to recreate the same models I executed back in December 2020, I will be utilizing the same packages in order to understand what my aim was for the project.

This is to improve the final results I had before to see if my problem statement still holds true.

In [1]:
# Import Packages
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.metrics import plot_confusion_matrix, f1_score, roc_curve

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report

import pickle